In [1]:
# Import useful packages
import os
import time
import pandas as pd
import glob
import numpy as np

# For loading/querying postgres
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import connect

In [2]:
# Create SQL engine to load data to postgres for central usage
from sqlalchemy import create_engine
dbname = '697_temp'
user = 'postgres'
password = 'poRter!5067'
host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com'

engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

### Loading Open Positions to Postgres

In [3]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Open_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Open_Positions_Data/joey_open_positions.csv']

In [4]:
# create temp_df to test loading; pick model not already in table
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
0,Positions_2020-07-20,2020-07-20,AMD,5404,305958.257475,1188.950975,306254.862300,1190.483450,-0.096943,75percent_confidence_no_holding_15m_cnn
1,Positions_2020-07-20,2020-07-20,BABA,1692,433770.791750,6388.849200,433576.962725,6385.678725,0.044685,75percent_confidence_no_holding_15m_cnn
2,Positions_2020-07-20,2020-07-20,BAC,19174,440440.421098,620.394942,440550.889512,620.478926,-0.025081,75percent_confidence_no_holding_15m_cnn
3,Positions_2020-07-20,2020-07-20,F,54710,366230.399675,160.750375,366137.472500,160.595100,0.025374,75percent_confidence_no_holding_15m_cnn
4,Positions_2020-07-20,2020-07-20,JNJ,1229,179156.380999,2477.942746,179041.428959,2476.408723,0.064163,75percent_confidence_no_holding_15m_cnn


In [7]:
# Initial dataframe needs to create a new table
temp_df.to_sql('open_positions', engine)

In [8]:
# Append data to table
temp_df.to_sql('open_positions', engine, if_exists='append')

In [11]:
# Test with a load
def import_open_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM open_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['key', 'model'])
    return data
open_pos_df = import_open_positions().drop('index',axis=1)

open_pos_df.tail()


,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
15262,Positions_2021-07-23,2021-07-23,MSFT,1660,479446.871675,7800.583525,479556.175825,7801.753800,-0.022798,75percent_confidence_no_holding_15m_cnn
15263,Positions_2021-07-23,2021-07-23,NVDA,3503,685190.240400,5661.758775,684601.710775,5658.773425,0.085893,75percent_confidence_no_holding_15m_cnn
15264,Positions_2021-07-23,2021-07-23,QQQ,1472,539411.405900,9167.625925,539449.534375,9168.497675,-0.007069,75percent_confidence_no_holding_15m_cnn
15265,Positions_2021-07-23,2021-07-23,SPY,1155,506127.561600,10965.594350,506116.372000,10965.540200,0.002211,75percent_confidence_no_holding_15m_cnn
15266,Positions_2021-07-23,2021-07-23,TSLA,470,302600.480325,10940.374600,302589.959650,10942.778600,0.003477,75percent_confidence_no_holding_15m_cnn


In [14]:
open_pos_df[open_pos_df['model']=='RF Reg_target_60_rebal_30_2017-01-01'].Date.max()

'2020-09-04'

### Loading track records to postgres

In [25]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("track_record/*.csv"):
    txtfiles.append(file)
    
txtfiles

['track_record/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'track_record/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'track_record/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [26]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

,Date,Value,Val_ex_cash,model
0,2020-08-06,100000.000000,99465.751610,RF Reg_target_60_rebal_30_2017-01-01
1,2020-08-07,101398.918613,100864.670223,RF Reg_target_60_rebal_30_2017-01-01
2,2020-08-10,103963.916785,103429.668394,RF Reg_target_60_rebal_30_2017-01-01
3,2020-08-11,105306.857052,104772.608661,RF Reg_target_60_rebal_30_2017-01-01
4,2020-08-12,104401.500965,103867.252575,RF Reg_target_60_rebal_30_2017-01-01


In [ ]:
# Initial dataframe needs to create a new table
temp_df.to_sql('track_record', engine)

In [27]:
# Append data to table
temp_df.to_sql('track_record', engine, if_exists='append')

In [28]:
# Test with a load
def import_track_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM track_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['Date', 'model'])
    return data
track_df = import_track_record().drop('index',axis=1)

track_df.head()


,Date,Value,Val_ex_cash,model
144,2020-07-19 00:00:00,100000.0000,100000.0000,75percent_confidence_no_holding_15m_cnn
145,2020-07-20 00:00:00,102126.2298,102126.2298,75percent_confidence_no_holding_15m_cnn
146,2020-07-21 00:00:00,101447.4265,101447.4265,75percent_confidence_no_holding_15m_cnn
147,2020-07-22 00:00:00,102702.7938,102702.7938,75percent_confidence_no_holding_15m_cnn
148,2020-07-23 00:00:00,102327.8937,102327.8937,75percent_confidence_no_holding_15m_cnn


### Loading cash records to postgres

In [29]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Cash_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Cash_Positions_Data/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'Cash_Positions_Data/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'Cash_Positions_Data/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [30]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.tail()

,key,cash,model
139,cash_2021-02-25,23.770446,RF Reg_target_60_rebal_30_2017-01-01
140,cash_2021-02-26,23.770446,RF Reg_target_60_rebal_30_2017-01-01
141,cash_2021-03-01,23.770446,RF Reg_target_60_rebal_30_2017-01-01
142,cash_2021-03-02,23.770446,RF Reg_target_60_rebal_30_2017-01-01
143,cash_2021-03-03,23.770446,RF Reg_target_60_rebal_30_2017-01-01


In [31]:
# Initial dataframe needs to create a new table
temp_df.to_sql('cash_record', engine, if_exists='append')

In [10]:
# Test with a load
def import_cash_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM cash_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
cash_df = import_cash_record().drop('index',axis=1)

cash_df.tail()


,key,cash,model
407,cash_2021-02-25,23.770446,RF Reg_target_60_rebal_30_2017-01-01
408,cash_2021-02-26,23.770446,RF Reg_target_60_rebal_30_2017-01-01
409,cash_2021-03-01,23.770446,RF Reg_target_60_rebal_30_2017-01-01
410,cash_2021-03-02,23.770446,RF Reg_target_60_rebal_30_2017-01-01
411,cash_2021-03-03,23.770446,RF Reg_target_60_rebal_30_2017-01-01


In [15]:
max_date = open_pos_df[open_pos_df['model']=='RF Reg_target_60_rebal_30_2017-01-01'].Date.max()

In [20]:
cash_df[(cash_df['key']==f'cash_{max_date}')&(cash_df['model']=='RF Reg_target_60_rebal_30_2017-01-01')].cash.values[0]

251.6393478362146

### Loading Value Positions to postgres

In [33]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Val_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Val_Positions_Data/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'Val_Positions_Data/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'Val_Positions_Data/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [34]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.tail()

,key,value,model
139,val_2021-02-25,131991.759044,RF Reg_target_60_rebal_30_2017-01-01
140,val_2021-02-26,128376.205474,RF Reg_target_60_rebal_30_2017-01-01
141,val_2021-03-01,129952.513408,RF Reg_target_60_rebal_30_2017-01-01
142,val_2021-03-02,130075.125108,RF Reg_target_60_rebal_30_2017-01-01
143,val_2021-03-03,130563.511682,RF Reg_target_60_rebal_30_2017-01-01


In [35]:
# Initial dataframe needs to create a new table
temp_df.to_sql('val_positions', engine,if_exists='append')

In [36]:
# Test with a load
def import_val_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM val_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
val_df = import_val_positions().drop('index',axis=1)

val_df.tail()

,key,value,model
777,val_2021-02-25,131991.759044,RF Reg_target_60_rebal_30_2017-01-01
778,val_2021-02-26,128376.205474,RF Reg_target_60_rebal_30_2017-01-01
779,val_2021-03-01,129952.513408,RF Reg_target_60_rebal_30_2017-01-01
780,val_2021-03-02,130075.125108,RF Reg_target_60_rebal_30_2017-01-01
781,val_2021-03-03,130563.511682,RF Reg_target_60_rebal_30_2017-01-01
